<a href="https://colab.research.google.com/github/thaisflopes/mestrado_PPGA_UnB/blob/main/modelos_ML/pre_processamento_modelos_ML_valor_total_condenacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Bibliotecas
import pandas as pd
import numpy as np

### Baixando bases de dados

In [ ]:
# Baixando a bases de dados de judicialização
data = pd.read_csv('dados_judicializacao_AED.csv')

In [ ]:
#Contabilizando a quantidade de linhas inicial
num_linhas_inicial = data.shape[0]

print("Número de linhas inicial:", num_linhas_inicial)

Número de linhas inicial: 555546


In [ ]:
#Visualizando dados
print('\n Tipos de dados do DataFrame:\n', data.dtypes)


 Tipos de dados do DataFrame:
 empresa_re                       object
num_proc_ano                      int64
data_distribuicao                object
UF                               object
operacao                         object
jurisdicao                       object
instancia                        object
situacao_processual_sentenca     object
n_autores                         int64
data_voo                         object
valor_pedido_inicial            float64
valor_dano_moral                float64
valor_dano_material             float64
valor_total_condenacao          float64
valor_pago                      float64
obj_problemas_operacionais         bool
obj_bagagem                        bool
obj_contrato                       bool
obj_outros                         bool
causa_nao_informado                bool
causa_forca_maior                  bool
causa_fortuitos                    bool
causa_terceiros                    bool
causa_cliente                      bool
causa_ae

### Preparando as variáveis para os modelos

Variáveis que não serão usadas: data_distribuicao, UF, data_voo, obj_principal, subobj_principal, causa_alegada, valor_pago

Variáveis com alterações:
- filtrar apenas dados em que a situacao_processual_sentenca é Condenação ou Acordo
- valor_pedido_inicial: transformar em Baixo, Médio e Alto
- valor_dano_moral: transformar em Baixo, Médio e Alto
- valor_dano_material: transformar em Baixo, Médio e Alto
- valor_total_condenacao: transformar em Baixo, Médio e Alto

Variáveis sem alterações: empresa_re, num_proc_ano, operacao, jurisdicao, instancia, situacao_processual_sentenca, n_autores, obj_problemas_operacionais, obj_bagagem, obj_contrato, obj_outros, causa_nao_informado, causa_forca_maior, causa_fortuitos, causa_terceiros, causa_cliente, causa_aerea, causa_outros, mes_distribuicao, regiao

#### Excluindo variáveis que não serão usadas

In [ ]:
# Excluindo colunas que não serão usadas
data = data.drop(['data_distribuicao', 'UF', 'data_voo', 'obj_principal', 'subobj_principal', 'causa_alegada', 'valor_pago'], axis=1)

#### Filtrando apenas as linhas em que a situacao_processual_sentenca é Condenação ou Acordo

In [ ]:
# Conferindo valores situacao_processual_sentenca
data.situacao_processual_sentenca.value_counts()

situacao_processual_sentenca
Outros                                       304091
Condenação                                   126041
Acordo                                        81266
Decisão favorável à aérea (improcedência)     38858
Extinção sem julgamento do mérito              5290
Name: count, dtype: int64

In [ ]:
# Filtrando as linhas onde situacao_processual_sentenca = Condenação
data = data[(data['situacao_processual_sentenca'] == 'Condenação') | (data['situacao_processual_sentenca'] == 'Acordo')]

# Calculando o mínimo e o máximo da coluna 'valor_pago'
minimo = data['valor_total_condenacao'].min()
maximo = data['valor_total_condenacao'].max()

print(f"Mínimo: {minimo}")
print(f"Máximo: {maximo}")

# Calculando a frequência de cada número na coluna 'valor_pago'
frequencia = data['valor_total_condenacao'].value_counts()

print(frequencia)

Mínimo: 0.0
Máximo: 1094059.0
valor_total_condenacao
0.00        82641
5000.00     10324
3000.00     10111
4000.00      8169
2000.00      6759
            ...  
8033.83         1
2444.44         1
5884.75         1
250.93          1
69277.59        1
Name: count, Length: 51874, dtype: int64


In [ ]:
#Contabilizando a quantidade de linhas após o filtro de situação processual e sentença
num_linhas_filtro = data.shape[0]

print("Número de linhas após o filtro de situação processual e sentença:", num_linhas_filtro)

Número de linhas após o filtro de situação processual e sentença: 207307


#### Indenizações

In [ ]:
#Removendo outliers (valores acima de 1 milhão)

#Lista de colunas para verificar os outliers
colunas_indenizacoes = ['valor_pedido_inicial', 'valor_dano_moral', 'valor_dano_material', 'valor_total_condenacao']

#Definindo valor máximo para remoção de outliers
valor_maximo = 1e6  # R$ 1.000.000

#Removendo as linhas que têm valores maiores que 1.000.000 nas colunas especificadas
data = data[~((data[colunas_indenizacoes] > valor_maximo).any(axis=1))]

#Contabilizando a quantidade de linhas após a remoção dos outliers
num_linhas_sem_outlier = data.shape[0]

print("Número de linhas após a remoção dos outliers:", num_linhas_sem_outlier)

Número de linhas após a remoção dos outliers: 207294


##### Valor pedido inicial

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_pedido_inicial'
percentil_25_pedido_inicial = data['valor_pedido_inicial'].quantile(0.25)
percentil_50_pedido_inicial = data['valor_pedido_inicial'].quantile(0.50)
percentil_75_pedido_inicial = data['valor_pedido_inicial'].quantile(0.75)

print(percentil_25_pedido_inicial, percentil_50_pedido_inicial, percentil_75_pedido_inicial)

10000.0 15000.0 20472.545000000002


In [ ]:
#Obtendo os valores únicos da coluna 'valor_pedido_inicial'
valores_unicos_pedido_inicial = data['valor_pedido_inicial'].dropna().unique()

#Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_pedido_inicial = np.min(valores_unicos_pedido_inicial)
percentil_25_pedido_inicial = np.percentile(valores_unicos_pedido_inicial, 25)
percentil_50_pedido_inicial = np.percentile(valores_unicos_pedido_inicial, 50)
percentil_75_pedido_inicial = np.percentile(valores_unicos_pedido_inicial, 75)
valor_maximo_pedido_inicial = np.max(valores_unicos_pedido_inicial)

#Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_pedido_inicial}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_pedido_inicial}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_pedido_inicial}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_pedido_inicial}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_pedido_inicial}")

Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 10332.64
Percentil 50 (com base nos valores únicos): 15322.92
Percentil 75 (com base nos valores únicos): 22281.32
Máximo (com base nos valores únicos): 950000.0


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados

condicoes = [
    pd.isnull(data['valor_pedido_inicial']),  # Nulos
    data['valor_pedido_inicial'] <= percentil_25_pedido_inicial,  # Baixo: até o percentil 25
    (data['valor_pedido_inicial'] > percentil_25_pedido_inicial) & (data['valor_pedido_inicial'] <= percentil_75_pedido_inicial),  # Médio: entre o percentil 25 e 75
    data['valor_pedido_inicial'] > percentil_75_pedido_inicial  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Substituindo os valores da coluna 'valor_pedido_inicial' com as categorias
data['valor_pedido_inicial'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_pedido_inicial'
data['valor_pedido_inicial'].value_counts()

valor_pedido_inicial
Médio    94899
Baixo    70221
Alto     42174
Name: count, dtype: int64

##### Dano Moral

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_moral'
percentil_25_dano_moral = data['valor_dano_moral'].quantile(0.25)
percentil_50_dano_moral = data['valor_dano_moral'].quantile(0.50)
percentil_75_dano_moral = data['valor_dano_moral'].quantile(0.75)

print(percentil_25_dano_moral, percentil_50_dano_moral, percentil_75_dano_moral)

0.0 0.0 2000.0


In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_moral' somente com os valores únicos

#Obtendo os valores únicos da coluna 'valor_dano_moral'
valores_unicos_dano_moral = data['valor_dano_moral'].dropna().unique()

#Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_dano_moral = np.min(valores_unicos_dano_moral)
percentil_25_dano_moral = np.percentile(valores_unicos_dano_moral, 25)
percentil_50_dano_moral = np.percentile(valores_unicos_dano_moral, 50)
percentil_75_dano_moral = np.percentile(valores_unicos_dano_moral, 75)
valor_maximo_dano_moral = np.max(valores_unicos_dano_moral)

#Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_dano_moral}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_dano_moral}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_dano_moral}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_dano_moral}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_dano_moral}")

Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 1520.935
Percentil 50 (com base nos valores únicos): 3236.54
Percentil 75 (com base nos valores únicos): 5970.86
Máximo (com base nos valores únicos): 465000.0


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados

condicoes = [
    pd.isnull(data['valor_dano_moral']),  # Nulos
    data['valor_dano_moral'] <= percentil_25_dano_moral,  # Baixo: até o percentil 25
    (data['valor_dano_moral'] > percentil_25_dano_moral) & (data['valor_dano_moral'] <= percentil_75_dano_moral),  # Médio: entre o percentil 25 e 75
    data['valor_dano_moral'] > percentil_75_dano_moral  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Substituindo os valores da coluna 'valor_dano_moral' com as categorias
data['valor_dano_moral'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_dano_moral'
data['valor_dano_moral'].value_counts()

valor_dano_moral
Baixo    151253
Médio     37675
Alto      18366
Name: count, dtype: int64

##### Dano Material

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_material'
percentil_25_dano_material = data['valor_dano_material'].quantile(0.25)
percentil_50_dano_material = data['valor_dano_material'].quantile(0.50)
percentil_75_dano_material = data['valor_dano_material'].quantile(0.75)

print(percentil_25_dano_material, percentil_50_dano_material, percentil_75_dano_material)

0.0 0.0 0.0


In [ ]:
# Obtendo os valores únicos da coluna 'valor_dano_material'
valores_unicos_dano_material = data['valor_dano_material'].dropna().unique()

# Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_dano_material = np.min(valores_unicos_dano_material)
percentil_25_dano_material = np.percentile(valores_unicos_dano_material, 25)
percentil_50_dano_material = np.percentile(valores_unicos_dano_material, 50)
percentil_75_dano_material = np.percentile(valores_unicos_dano_material, 75)
valor_maximo_dano_material = np.max(valores_unicos_dano_material)

# Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_dano_material}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_dano_material}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_dano_material}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_dano_material}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_dano_material}")

Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 450.605
Percentil 50 (com base nos valores únicos): 1107.85
Percentil 75 (com base nos valores únicos): 2638.105
Máximo (com base nos valores únicos): 453880.56


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados

condicoes = [
    pd.isnull(data['valor_dano_material']),  # Nulos
    data['valor_dano_material'] <= percentil_25_dano_material,  # Baixo: até o percentil 25
    (data['valor_dano_material'] > percentil_25_dano_material) & (data['valor_dano_material'] <= percentil_75_dano_material),  # Médio: entre o percentil 25 e 75
    data['valor_dano_material'] > percentil_75_dano_material  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Substituindo os valores da coluna 'valor_dano_material' com as categorias
data['valor_dano_material'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_dano_material'
data['valor_dano_material'].value_counts()

valor_dano_material
Baixo    181275
Médio     17482
Alto       8537
Name: count, dtype: int64

##### Valor total da condenação

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_material'
percentil_25_condenacao = data['valor_total_condenacao'].quantile(0.25)
percentil_50_condenacao = data['valor_total_condenacao'].quantile(0.50)
percentil_75_condenacao = data['valor_total_condenacao'].quantile(0.75)

print(percentil_25_condenacao, percentil_50_condenacao, percentil_75_condenacao)

0.0 1813.93 5000.0


In [ ]:
# Obtendo os valores únicos da coluna 'valor_condenacao'
valores_unicos_condenacao = data['valor_total_condenacao'].dropna().unique()

# Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_condenacao = np.min(valores_unicos_condenacao)
percentil_25_condenacao = np.percentile(valores_unicos_condenacao, 25)
percentil_50_condenacao = np.percentile(valores_unicos_condenacao, 50)
percentil_75_condenacao = np.percentile(valores_unicos_condenacao, 75)
valor_maximo_condenacao = np.max(valores_unicos_condenacao)

# Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_condenacao}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_condenacao}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_condenacao}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_condenacao}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_condenacao}")

Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 1729.02
Percentil 50 (com base nos valores únicos): 4051.415
Percentil 75 (com base nos valores únicos): 7236.8175
Máximo (com base nos valores únicos): 951084.0


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados
condicoes = [
    pd.isnull(data['valor_total_condenacao']),  # Nulos
    data['valor_total_condenacao'] <= percentil_25_condenacao,  # Baixo: até o percentil 25
    (data['valor_total_condenacao'] > percentil_25_condenacao) & (data['valor_total_condenacao'] <= percentil_75_condenacao),  # Médio: entre o percentil 25 e 75
    data['valor_total_condenacao'] > percentil_75_condenacao  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Substituindo os valores da coluna 'valor_condenacao' com as categorias
data['valor_total_condenacao'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_condenacao'
data['valor_total_condenacao'].value_counts()

valor_total_condenacao
Baixo    103018
Médio     77477
Alto      26799
Name: count, dtype: int64

In [ ]:
#Criando arquivo csv
data.to_csv('dados_modelos_ML.csv', index=False, encoding='utf-8')